In [1]:
project_name = "BUName_usecase"
env = 'dev'
job_name = 'inf'

In [2]:
import os
import pickle
import boto3
import gzip
from io import BytesIO
#import joblib
import sagemaker
import sagemaker.session

from sagemaker.transformer import Transformer
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput,TransformInput,CreateModelInput
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.conditions import (
    ConditionGreaterThanOrEqualTo,
)
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)
from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
)
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import (
    ProcessingStep,
    TrainingStep,
    TransformStep,
    CreateModelStep
)
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.model import Model
from sagemaker.network import NetworkConfig

from datetime import datetime
from dateutil import tz
import pytz
import json
from sagemaker.workflow.parameters import (
        ParameterInteger,
        ParameterString,
    )


sess = sagemaker.Session()


def convert_timezone():
    to_zone = tz.gettz('Asia/Kolkata')
    
    return datetime.utcnow().astimezone(to_zone)

   
def get_session(region, default_bucket):
    """Gets the sagemaker session based on the region.
    Args:
        region: the aws region to start the session
        default_bucket: the bucket to use for storing the artifacts
    Returns:
        `sagemaker.session.Session instance
    """

    boto_session = boto3.Session(region_name=region)

    sagemaker_client = boto_session.client("sagemaker")
    runtime_client = boto_session.client("sagemaker-runtime")
    return sagemaker.session.Session(
        boto_session=boto_session,
        sagemaker_client=sagemaker_client,
        sagemaker_runtime_client=runtime_client,
        default_bucket=default_bucket,
    )

def latest_config():
        
        '''
        this will pick the latest trained config file
        '''

        s3_path = f"s3://dlk-cloud-tier-8-code-ml-{env}/"
        config_bucket = f"dlk-cloud-tier-8-code-ml-{env}"
        s3 = boto3.client('s3')
        all_objects = s3.list_objects(Bucket = f"dlk-cloud-tier-8-code-ml-{env}", Prefix=f"config_files/pipeline_{job_name}_config/{project_name}/")
        model_folder=all_objects['Contents'][-1]['Key']
        #model_path=s3_path+model_folder

        return config_bucket, model_folder

config_s3_bucket = latest_config()[0]
model_s3_path = latest_config()[1]


def getJsonData(bucket_name,key_name):
        '''
        this will pick the json config file from s3 bucket
        '''

        print("[LOG]", bucket_name,'---------')
        print("[LOG]", key_name,'--------------')

        s3_client = boto3.client('s3')
        csv_obj = s3_client.get_object(Bucket=bucket_name, Key=key_name)

        body = csv_obj['Body']

        json_string = body.read().decode('utf-8')
        json_content = json.loads(json_string)

        return json_content


config = getJsonData(config_s3_bucket,model_s3_path)


def model_with_pipeline():

    env='dev'
    s3_path = f"s3://dlk-cloud-tier-9-training-ml-{env}/"
    s3 = boto3.client('s3')
    all_objects = s3.list_objects(Bucket = f"dlk-cloud-tier-9-training-ml-{env}", Prefix=f"{project_name}/models/")
    model_folder=all_objects['Contents'][-1]['Key']
    model_path=s3_path+model_folder
    return model_path

### Full pipeline

In [7]:
def get_pipeline(
    region,
    subnets,
    security_group_ids,
    role=None,
    default_bucket=None,
    model_package_group_name=config['pipeline']['pipeline_config']['model_package_group_name'],
    pipeline_name=config['pipeline']['pipeline_config']['pipeline_name'],  # You can find your pipeline name in the Studio UI (project -> Pipelines -> name)
    base_job_prefix=config['pipeline']['pipeline_config']['base_job_prefix'],
    env='dev'):
    
    srilanka_tz = pytz.timezone('Asia/Colombo')
    s3 = boto3.client('s3')
    #working input data path
    input_data = config['preprocessing']['inputs']['input1']
    #input_data2 = config['preprocessing']['inputs']['input2']
    #input_data3 = config['preprocessing']['inputs']['input3']
    #input_data4 = config['preprocessing']['inputs']['input4']

    directory_name = config['pipeline']['pipeline_config']['base_job_prefix']
    date_folder = datetime.now(srilanka_tz).strftime("%Y-%m-%d")
       
    sagemaker_session = get_session(region, default_bucket)
    if role is None:
        role = sagemaker.session.get_execution_role(sagemaker_session)
    account_id = boto3.client("sts").get_caller_identity().get("Account")
    region = boto3.session.Session().region_name

    # Parameters for pipeline execution
    
    preprocess_output_data1 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/{job_name}/{date_folder}/preprocess_output_data/output1"
    #preprocess_output_data2 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}{project_name}/{job_name}/{date_folder}/preprocess_output_data/output2"
    #preprocess_output_data3 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/{job_name}/{date_folder}/preprocess_output_data/output3"
    #preprocess_output_data4 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/{job_name}/{date_folder}/preprocess_output_data/output4"
    
    postprocess_output_data1 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/{job_name}/{date_folder}/postprocess_output_data/post-output1"
    #postprocess_output_data2 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}{project_name}/{job_name}/{date_folder}/postprocess_output_data/post-output2"
    #postprocess_output_data3 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/{job_name}/{date_folder}/postprocess_output_data/post-output3"
    #postprocess_output_data4 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/{job_name}/{date_folder}/postprocess_output_data/post-output4"
    
    
    model_approval_status = ParameterString(
        name="ModelApprovalStatus",
        default_value="Approved",  # ModelApprovalStatus can be set to a default of "Approved" if you don't want manual approval.
    )

    generic_tags=[{'Key': 'EnvironmentName', 'Value': env}, 
                  {'Key': 'ProjectName', 'Value': config['pipeline']['generic_tags']['ProjectName']},
                  {'Key': 'DepartmentName', 'Value': config['pipeline']['generic_tags']['DepartmentName']}, 
                  {'Key': 'UsecaseName', 'Value': config['pipeline']['generic_tags']['UsecaseName']},
                  {'Key': 'ResourceName', 'Value': config['pipeline']['generic_tags']['ResourceName']}, 
                  {'Key': 'OwnerName', 'Value': config['pipeline']['generic_tags']['OwnerName']},
                  {'Key': 'BUName', 'Value': config['pipeline']['generic_tags']['BUName']} ]
    
    
    
    # --------------------- PREPROCESSING --------------------------------------------------------------------
    ecr_repository = config['preprocessing1']['estimator_config']['ecr_repository']
    tag = ":latest"
    uri_suffix = "amazonaws.com"
    
    preprocessing_repository_uri = "{}.dkr.ecr.{}.{}/{}".format(
        account_id, region, uri_suffix, ecr_repository + tag
    )
        
    script_processor = ScriptProcessor(
         command = ["python3"],
         image_uri = preprocessing_repository_uri,
         role = role,
         instance_count = config['preprocessing1']['estimator_config']['instance_count'],
         instance_type = config['preprocessing1']['estimator_config']['instance_type'],
         tags = generic_tags + [{'Key': 'JobType', 'Value': 'Preprocessing'}],
         network_config = NetworkConfig(subnets=subnets.split(':'), security_group_ids=security_group_ids.split(':'))
    )
    
    step_preprocess = ProcessingStep(
        name=f"{base_job_prefix}-initialpreprocessing",
        processor=script_processor, 
        code=config['preprocessing1']['inputs']['code'],
        inputs=[ProcessingInput(input_name="input1",source=input_data, destination="/opt/ml/processing/input1"),
                #ProcessingInput(input_name="input2",source=input_data, destination="/opt/ml/processing/input2"),
                #ProcessingInput(input_name="input3",source=input_data, destination="/opt/ml/processing/input3"),
                #ProcessingInput(input_name="input4",source=input_data, destination="/opt/ml/processing/input4"),
               ],
        outputs=[
            ProcessingOutput(output_name="output1", destination=preprocess_output_data1, source="/opt/ml/processing/output1"),
            ProcessingOutput(output_name="output2", destination=preprocess_output_data2, source="/opt/ml/processing/output2"),
            #ProcessingOutput(output_name="output4", destination=preprocess_output_data3, source="/opt/ml/processing/output3"),
            #ProcessingOutput(output_name="output4", destination=preprocess_output_data4, source="/opt/ml/processing/output4"),
        ]
        #job_arguments=["--env", env],
    )
    
    ##################################### Inference Job ##############################
    ecr_repository_inf_dt = config['preprocessing1']['estimator_config']['ecr_repository']
    tag = ":latest"
    uri_suffix = "amazonaws.com"
    
    preprocessing_repository_uri = "{}.dkr.ecr.{}.{}/{}".format(
        account_id, region, uri_suffix, ecr_repository + tag
    )
    
    BU = config['pipeline']['pipeline_config']['BUName']
    prj_name = config['pipeline']['pipeline_config']['project_name']

    step_create_model = CreateModelStep(
        name=f"{base_job_prefix}-createdecisiontreemodel",
        model=Model( 
            image_uri = f"120582440665.dkr.ecr.ap-southeast-1.amazonaws.com/{BU}-{prj_name}-inf-inferencedt-image:latest",
                    model_data=model_with_pipeline(),
                    role = role,
                    sagemaker_session = sagemaker_session,
                    name = f"{base_job_prefix}",
                    vpc_config = {'Subnets':subnets.split(':'),
                                    'SecurityGroupIds': security_group_ids.split(':')}),
        inputs=CreateModelInput(instance_type=config['create_model-dt']['estimator_config']['instance_type'])
    )
    
    model_name = step_create_model.properties.ModelName
    
    transformer = Transformer(model_name=model_name,
                              instance_count=1,
                              #instance_type=instance_type,
                              #strategy='SingleRecord',
                              strategy='MultiRecord',
                              #max_payload=10, 
                              assemble_with="Line",
                              max_concurrent_transforms = 100,
                              instance_type=config['inference-dt']['estimator_config']['instance_type'],
                              output_path=f"s3://dlk-cloud-tier-11-inference-ml-{env}/{project_name}/batch/{date_folder}/",
                              accept='text/csv',
                              tags = generic_tags + [{'Key': 'JobName', 'Value': 'Inference'}]
                )
    
    step_transform = TransformStep(
        name=f"{base_job_prefix}-inference",
        transformer=transformer,
        inputs=TransformInput(
            data = step_preprocess.properties.ProcessingOutputConfig.Outputs["output1"].S3Output.S3Uri,
            split_type="Line",
            content_type="text/csv",
            #compression_type = 'Gzip', 
            #input_filter='$[1:]', 
            join_source='Input')
            #output_filter='$[0,-1]')
    )
   
    # --------------------- POST-PROCESSING --------------------------------------------------------------------
    
    script_postprocess = ScriptProcessor(
         command = ["python3"],
         image_uri = preprocessing_repository_uri,
         role = role,
         instance_count = config['preprocessing2']['estimator_config']['instance_count'],
         instance_type = config['preprocessing2']['estimator_config']['instance_type'],
         tags = generic_tags + [{'Key': 'JobType', 'Value': 'Postprocessing'}],
         network_config = NetworkConfig(subnets=subnets.split(':'), security_group_ids=security_group_ids.split(':'))
    )
    
    step_postproces = ProcessingStep(
        name=f"{base_job_prefix}-postprocessing",
        processor=script_postprocess, 
        code=config['postpreprocessing']['inputs']['code'],
        inputs=[ProcessingInput(input_name="post-input1",
                                source=step_transform.properties.TransformOutput.S3OutputPath, 
                                destination="/opt/ml/processing/post-input1",
                               s3_data_type='S3Prefix'),
               #ProcessingInput(input_name="post-input2", 
                #               source= step_preprocess.properties.ProcessingOutputConfig.Outputs["input2"].S3Output.S3Uri, 
                 #              destination= "/opt/ml/processing/post-input2"),
                               #ProcessingInput(input_name="post-input3", 
                #               source= step_preprocess.properties.ProcessingOutputConfig.Outputs["input3"].S3Output.S3Uri, 
                 #              destination= "/opt/ml/processing/post-input3"),
               ],
        outputs=[
            ProcessingOutput(output_name="post-output1", destination=post_output_data, source="/opt/ml/processing/post-output1"),
            #ProcessingOutput(output_name="post-output2", destination=post_output_data, source="/opt/ml/processing/post-output2"),
            #ProcessingOutput(output_name="post-output3", destination=post_output_data, source="/opt/ml/processing/post-output3"),
            #ProcessingOutput(output_name="post-output4", destination=post_output_data, source="/opt/ml/processing/post-output4"),
        ]
        #job_arguments=["--env", env],
    )
        
    # Pipeline instance
    pipeline = Pipeline(
        name=pipeline_name+env,
        parameters=[
            env,
            model_approval_status,
        ],
        steps=[step_preprocess,
               step_create_model,
               step_transform, 
               step_postproces
              ],
        sagemaker_session=sagemaker_session,
    )
    return pipeline

In [8]:
#new subnet
dev_subnets = 'subnet-036d6b39301b4a41a'
dev_sg = 'sg-0970930479bbef573'

region ='ap-southeast-1'
role='arn:aws:iam::120582440665:role/Sagemaker'
#role=None
default_bucket='pipeline-sagemaker-test'
pipeline_def = get_pipeline(region, dev_subnets, dev_sg, role,default_bucket, env='dev')

pipeline_def.upsert(role_arn=role)

execution = pipeline_def.start()

In [11]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-southeast-1:120582440665:pipeline/mobile-prepaidchurn-inf-pipeline-dev',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-southeast-1:120582440665:pipeline/mobile-prepaidchurn-inf-pipeline-dev/execution/g3lsd606w2u1',
 'PipelineExecutionDisplayName': 'execution-1647314034366',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'mobile-prepaidchurn-inf-pipeline-dev',
  'TrialName': 'g3lsd606w2u1'},
 'CreationTime': datetime.datetime(2022, 3, 15, 3, 13, 54, 265000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 3, 15, 3, 13, 54, 265000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '0dbd5fb8-7508-44a1-9d1b-bf1d327c960d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0dbd5fb8-7508-44a1-9d1b-bf1d327c960d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '559',
   'date': 'Tue, 15 Mar 2022 03:13:53 GMT'},
  'Retry